## Adding Relationships

Our tables are now fully defined as we had discussed in the previous notebooks.
However, we can also use SQLModel to add additional relationships between tables.

Although not discussed in the previous notebook, if we wanted to query our previous database for an article and access the keywords,
we would have to either do two queries or we would need to do a SQL `join` operation.
However, when using an ORM, we can define additional relationships on the tables that allow us to access this more easily

In this next section, we'll add relationships to our SQLModel models.
To get started, we'll remove our old database and get started with a new one

The cell below redefines our associative tables first, then repeats our `Author` definition.
A final line is added to both `Article` and `Author` that defines a relationship between articles and authors.

```
authors: List["Author"] = Relationship(back_populates="articles", link_model=ArticleAuthor)
```

This defines a `relationship` on the `article` table. It says when we access an article we should get a list of `Author` objects with the article.
We are also telling the ORM that these two are linked using the `AriticleAuthor` table.

Now when working with our objects we can use `Article.authors` or `Author.articles` to reference across tables.



In [1]:
import os 

from typing import Optional, List

from sqlmodel import Field, SQLModel, Session, Relationship, create_engine

In [2]:
class ArticleKeyword(SQLModel, table=True):
    __table_args__ = {"extend_existing": True} # This lets us run the Jupyter notebook cell multiple times without error
    
    article_doi: str = Field(foreign_key="article.doi", primary_key=True)
    keyword_id: str = Field(foreign_key="keyword.id", primary_key=True)

class ArticleAuthor(SQLModel, table=True):
    __table_args__ = {"extend_existing": True}

    article_doi: str = Field(foreign_key="article.doi", primary_key=True)
    author_id: str = Field(foreign_key="author.id", primary_key=True)


class Article(SQLModel, table=True):
    __table_args__ = {"extend_existing": True}
    
    doi: str = Field(primary_key=True)
    title: str
    publication_year: int
    abstract: Optional[str] = Field(default=None)

    authors: List["Author"] = Relationship(back_populates="articles", link_model=ArticleAuthor)

class Author(SQLModel, table=True):
    __table_args__ = {"extend_existing": True}

    id: None | int = Field(primary_key=True)
    first_name: str
    last_name: str
    affiliation: Optional[str] = Field(default=None)

    articles: List["Article"] = Relationship(back_populates="authors", link_model=ArticleAuthor)


<div class="alert alert-block alert-warning">

## Exercise
Redefine the Keyword table and add a relationship with `Article`.
</div>

In [3]:
class Article(SQLModel, table=True):
    __table_args__ = {"extend_existing": True}
    
    doi: str = Field(primary_key=True)
    title: str
    publication_year: int
    abstract: Optional[str] = Field(default=None)

    authors: List["Author"] = Relationship(back_populates="articles", link_model=ArticleAuthor)

## Add Keyword with relationship here


class Keyword(SQLModel, table=True):
    __table_args__ = {"extend_existing": True}

    id: None | int = Field(primary_key=True)
    keyword: str = Field(unique=True, index=True)

    articles: list["Article"] = Relationship(back_populates="keywords", link_model=ArticleKeyword)

/home/janash/miniconda3/envs/database-tutorial/lib/python3.12/site-packages/sqlmodel/main.py:638: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Article, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


In [5]:
sqlite_file_name = "sqlmodel_database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url, echo=True)

SQLModel.metadata.create_all(engine)

2024-08-28 04:51:11,862 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-28 04:51:11,870 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("articlekeyword")
2024-08-28 04:51:11,875 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-28 04:51:11,876 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("articlekeyword")
2024-08-28 04:51:11,887 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-28 04:51:11,888 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("articleauthor")
2024-08-28 04:51:11,889 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-28 04:51:11,890 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("articleauthor")
2024-08-28 04:51:11,891 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-28 04:51:11,891 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("article")
2024-08-28 04:51:11,892 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-28 04:51:11,893 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("article")
2024-08-28 04:51:11,893 INFO sqlalchemy.

2024-08-28 04:51:11,929 INFO sqlalchemy.engine.Engine 
CREATE TABLE author (
	id INTEGER NOT NULL, 
	first_name VARCHAR NOT NULL, 
	last_name VARCHAR NOT NULL, 
	affiliation VARCHAR, 
	PRIMARY KEY (id)
)


2024-08-28 04:51:11,930 INFO sqlalchemy.engine.Engine [no key 0.00088s] ()
2024-08-28 04:51:11,939 INFO sqlalchemy.engine.Engine 
CREATE TABLE keyword (
	id INTEGER NOT NULL, 
	keyword VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


2024-08-28 04:51:11,939 INFO sqlalchemy.engine.Engine [no key 0.00065s] ()
2024-08-28 04:51:11,948 INFO sqlalchemy.engine.Engine CREATE UNIQUE INDEX ix_keyword_keyword ON keyword (keyword)
2024-08-28 04:51:11,949 INFO sqlalchemy.engine.Engine [no key 0.00101s] ()
2024-08-28 04:51:11,960 INFO sqlalchemy.engine.Engine 
CREATE TABLE articlekeyword (
	article_doi VARCHAR NOT NULL, 
	keyword_id VARCHAR NOT NULL, 
	PRIMARY KEY (article_doi, keyword_id), 
	FOREIGN KEY(article_doi) REFERENCES article (doi), 
	FOREIGN KEY(keyword_id) REFERENCES keyword (id)
)


2024-08-2